## <font color=green> **Imports**</font>

In [1]:
%load_ext autoreload
%autoreload 2
from imports import *
from utils import *
from stats import *
from tmstools import *
plt.ioff()

## <font color=green> **Reading MEP files**</font>

In [2]:
# nSide - number of stimulation points along a side of a square grid
# cellSize - the distance between two adjacent stimulation points
# iX, iY - indices of coordinate axes used in the indexing of stimulation point locations

nSide = 7; cellSize = 7.63; cellArea = cellSize**2; iX=0; iY=2;

In [3]:
# read TMS mapping data from the file 'grid mappings.csv'
# file format: index of subject, index of session, x, y, z (in mm), amplitude (in uV)
# x, y, z - coordinates of the electric field maximum at the selected peeling depth

data = np.loadtxt('grid mappings.csv', delimiter = ';')

In [4]:
iSubjs = (np.unique(data[:,0])-1).astype(int)
nSubj = len(iSubjs)
iSessions = np.array(range(3))

In [5]:
# make TMSmap objects from the lines of the file
TMSmaps = [[TMSmap(curLines[:,2:5], curLines[:,5], nSide, iX, iY)
             for curLines in [np.array([line for line in data if (line[0]==iSubj+1 and line[1] == iSess+1)])
             for iSess in iSessions]]
             for iSubj in iSubjs]

In [6]:
# check deviations of the number of stimuli per cell due to operator error
nStimCell = np.array([len(arr) for subj in TMSmaps for sess in subj for line in sess.ampl for arr in line])
freqs, bins = np.histogram(nStimCell, density = True, bins = np.arange(-0.5,15.5,1))
[[i,np.round(freqs[bins[:-1]==i-0.5][0],4)] for i in range(6,14)]

[[6, 0.0],
 [7, 0.0009],
 [8, 0.0043],
 [9, 0.0697],
 [10, 0.8861],
 [11, 0.0374],
 [12, 0.0017],
 [13, 0.0]]

## <font color=green> **Effect of grid size on representation coverage**</font>

In [3]:
# read fractions of motor representations not covered by grids of different sizes
# precomputed in the file 'fractions.xlsx'
sheet = xlrd.open_workbook('Map coverage/fractions.xlsx').sheet_by_index(0)
fracNotCov = np.zeros((10,121))
for nCells in [5,6,7,8,9]:
    fracNotCov[nCells] = np.array(sheet.col_values(3+(nCells-5)*3)[1:])
nMaps = len(fracNotCov[5])
sheet1 = xlrd.open_workbook('Map coverage/desc.xlsx').sheet_by_index(0)
muscle = np.array(sheet1.col_values(1)[1:])

In [5]:
# plot the fraction of all maps for which at least a given percentage of the representation
# is covered by a grid of a given size
for nCells in [5,6,7,8,9]:
    percentageCovered = 100 * (1-np.sort(fracNotCov[nCells])[::-1])
    fractionOfMaps = 1-np.arange(0,nMaps)/nMaps
    plt.plot(percentageCovered, fractionOfMaps)
plt.legend(['Grid side %2.0f mm, mean coverage: %2.1f%%'%(cellSize * nCells, 100*(1-np.mean(fracNotCov[nCells])))
            for nCells in [5,6,7,8,9]])
plt.xlabel('Percentage of representation covered'); plt.ylabel('Fraction of all maps');
plt.grid(); plt.xlim(50,100); plt.ylim(0.3,1); setSize(5,5)
plt.xticks(np.arange(50,105,5));plt.yticks(np.arange(0.3,1.05,0.05));
plt.savefig('01 Percentages of representations covered by grids.png', dpi = 300, bbox_inches = 'tight')
plt.close(plt.gcf())

In [9]:
# testing for differences between muscles
for nCells in [5,6,7,8,9]:
    print(scipy.stats.kruskal(fracNotCov[nCells,muscle=='FDS'],fracNotCov[nCells,muscle=='EDC'],fracNotCov[nCells,muscle=='APB']))

KruskalResult(statistic=0.33726825300464724, pvalue=0.8448179433436528)
KruskalResult(statistic=1.2667252147856558, pvalue=0.5308039115492833)
KruskalResult(statistic=0.674343042681281, pvalue=0.7137863998022502)
KruskalResult(statistic=0.8557865116182662, pvalue=0.6518809955944094)
KruskalResult(statistic=0.5338159629205024, pvalue=0.7657435308250692)


## <font color=green> **Comparisons of MEPs within and between sessions**</font>

In [10]:
# compare the MEP amplitudes in each grid cell:
# 1) the first 5 amplitudes in each of two sessions, for 3 pairs of sessions (between sessions)
# 2) the first and the last 5 amplitudes in each session (within a session)
pairs = np.array([[0,1],[1,2],[0,2]])
pBetweenSess = np.zeros((nSubj, 3, nSide, nSide))
signDiffBetweenSess = np.zeros((nSubj, 3, nSide, nSide))
pWithinSess = np.zeros((nSubj, 3, nSide, nSide))
signDiffWithinSess = np.zeros((nSubj, 3, nSide, nSide))
pThresh = 0.05
for iSubj in range(nSubj):
    for iPair,pair in enumerate(pairs):
        for i in range(nSide):
            for j in range(nSide):
                arr1 = censorSmall(TMSmaps[iSubj][pair[0]].ampl[i,j])
                arr2 = censorSmall(TMSmaps[iSubj][pair[1]].ampl[i,j])
                [p, stat] = pAndStatGehanR(arr1[:5], arr2[:5])  # compare first halves of sessions
                #[p, stat] = pAndStatGehanR(arr1[5:], arr2[5:])  # compare second halves of sessions
                pBetweenSess[iSubj,iPair,i,j] = p
                signDiffBetweenSess[iSubj,iPair,i,j] = (1 if stat > 0 else -1) if p < pThresh else 0
for iSubj in range(nSubj):
    for iSess in range(3):
        for i in range(nSide):
            for j in range(nSide):
                arr1 = censorSmall(TMSmaps[iSubj][iSess].ampl[i,j][:5])
                arr2 = censorSmall(TMSmaps[iSubj][iSess].ampl[i,j][5:])
                [p, stat] = pAndStatGehanR(arr1, arr2)
                pWithinSess[iSubj,iSess,i,j] = p
                signDiffWithinSess[iSubj,iSess,i,j] = (1 if stat > 0 else -1) if p < pThresh else 0

In [11]:
# number of significant differences between sessions
np.sum(abs(signDiffBetweenSess))

57.0

In [12]:
# Fisher's p-value synthesis to test the null hypothesis that there is no change in any of the cells.
# The test is anti-concervative for positively dependent tests
# (i.e. if MEPs in different cells are positively correlated),
# so the true combined p-value is larger than the calculated result.
scipy.stats.combine_pvalues([x for plane in pBetweenSess[:,1] for line in plane for x in line if ~np.isnan(x)])[1]

2.4045471980588395e-11

In [13]:
# number of significant differences within sessions
np.sum(abs(signDiffWithinSess))

12.0

In [14]:
# Fisher's p-value synthesis for the within-session changes
scipy.stats.combine_pvalues([x for plane in pWithinSess[:,1] for line in plane for x in line if ~np.isnan(x)])[1]

0.014635112568913795

In [15]:
colors = [(0, 0, 1), (0.5, 0.5, 0.5), (1, 0, 0)] 
cMap = [(value/2, colour) for value, colour in zip(range(3),colors)]
customColourMap = matplotlib.colors.LinearSegmentedColormap.from_list("custom", cMap)

In [71]:
f,ax = plt.subplots(nSubj,12)
for iSubj in range(nSubj):
    if iSubj == 0:
        i = 1
        ax[iSubj,i].set_title('Sess. 1',fontsize = 12); i+=1
        ax[iSubj,i].set_title('Sess. 2',fontsize = 12); i+=1
        ax[iSubj,i].set_title('Sess. 3',fontsize = 12); i+=2
        ax[iSubj,i].set_title('Diff. 1-2',fontsize = 12); i+=1
        ax[iSubj,i].set_title('Diff. 2-3',fontsize = 12); i+=1
        ax[iSubj,i].set_title('Diff. 1-3',fontsize = 12); i+=2
        ax[iSubj,i].set_title('Diff. 1',fontsize = 12); i+=1
        ax[iSubj,i].set_title('Diff. 2',fontsize = 12); i+=1
        ax[iSubj,i].set_title('Diff. 3',fontsize = 12); i+=1
    i = 0
    ax[iSubj,i].text(0.4,0.5,'Subj %d'%(iSubj+1), fontsize=10); i+=1
    ax[iSubj,i].imshow(TMSmaps[iSubj][0].pPositive()[:,::-1], vmin=0, vmax=1); i+=1
    ax[iSubj,i].imshow(TMSmaps[iSubj][1].pPositive()[:,::-1], vmin=0, vmax=1); i+=1
    ax[iSubj,i].imshow(TMSmaps[iSubj][2].pPositive()[:,::-1], vmin=0, vmax=1); i+=2
    ax[iSubj,i].imshow(signDiffBetweenSess[iSubj,0][:,::-1], cmap = customColourMap, vmin=-1, vmax=1); i+=1
    ax[iSubj,i].imshow(signDiffBetweenSess[iSubj,1][:,::-1], cmap = customColourMap, vmin=-1, vmax=1); i+=1
    ax[iSubj,i].imshow(signDiffBetweenSess[iSubj,2][:,::-1], cmap = customColourMap, vmin=-1, vmax=1); i+=2
    ax[iSubj,i].imshow(signDiffWithinSess[iSubj,0][:,::-1], cmap = customColourMap, vmin=-1, vmax=1); i+=1
    ax[iSubj,i].imshow(signDiffWithinSess[iSubj,1][:,::-1], cmap = customColourMap, vmin=-1, vmax=1); i+=1
    ax[iSubj,i].imshow(signDiffWithinSess[iSubj,2][:,::-1], cmap = customColourMap, vmin=-1, vmax=1); i+=1
ax1 = ax[0,2].imshow(TMSmaps[0][2].pPositive()[:,::-1], vmin=0, vmax=1)
cbaxes = plt.gcf().add_axes([0.1, 0.125, 0.01, 0.4]) 
cb = plt.colorbar(ax1, cax = cbaxes)
cbaxes.set_title('Fraction\nof MEPs\n>50 $\mu$V\n')
[ax1.axis('off') for line in ax for ax1 in line]
setSize(12,8)
fs = 12; y = 2.05; va = 'center'
plt.text(15, y, 'A. Maps', fontsize=fs, verticalalignment=va)
plt.text(33, y, 'B. Differences between sessions', fontsize=fs, verticalalignment=va)
plt.text(60, y, 'C. Differences within sessions\n(between halves of a session)', fontsize=fs, verticalalignment=va)
plt.savefig('02 Sessions & diff between and within.png', dpi = 300, bbox_inches='tight')
plt.close(plt.gcf())

## <font color=green> **Bootstrapping**</font>

In [17]:
# generate maps by bootstrapping (with replacement) from each session of each subject
nBootSamples = 1000
nStimBoot = 10
boots = np.array([[[randMapK(sess, nStimBoot) for sess in subj] for i in range(nBootSamples)] for subj in TMSmaps])

In [18]:
nStims = np.array(range(1,nStimBoot+1))

## <font color=green> **Bias of the area of the cells with at least half suprathreshold MEPs - illustration of non-monotonic dependence on the number of stimuli**</font>

In [19]:
def pBinom(pThr,nStim,pTrue):
# probability that a cell with the probability pTrue of suprathreshold MEP,
# being stimulated by nStim stimuli, will produce at least pThr*nStim suprathreshold MEPs
    return 1 - scipy.stats.binom.cdf(pThr*nStim,nStim,pTrue)

In [72]:
# illustration of non-monotonic dependence of the bias on the number of stimuli:
# dependence of the bias of the area of cells with more than half suprathreshold MEPs
# for a model representation with two cells,
# with probabilities of suprathreshold MEPs equal to 0.4 and 0.7
pThr = 0.5; pTrue = np.array([0.4,0.7]); areaTrue = np.sum(pTrue>pThr); nStims0 = np.arange(1,100,1)
meanAreas = np.array([np.sum([pBinom(pThr,nStim, p) for p in pTrue]) for nStim in nStims0])
biases = meanAreas/areaTrue - 1
f,ax = plt.subplots(3,1)
ax[0].plot(nStims0, biases); ax[0].set_title('A. Bias of the area for a model map with two cells')
ax[0].grid(); ax[0].set_xlabel('Number of stimuli per cell'); ax[0].set_ylabel('Normalized bias')
meanAreasOfCell1 = np.array([pBinom(pThr,nStim, pTrue[0]) for nStim in nStims0])
ax[1].plot(nStims0, meanAreasOfCell1); ax[1].set_title('B. Mean contribution of the 1st cell to the area, p(MEP) = 0.4')
ax[1].grid(); ax[1].set_xlabel('Number of stimuli per cell'); ax[1].set_ylabel('Area estimate')
meanAreasOfCell2 = np.array([pBinom(pThr,nStim, pTrue[1]) for nStim in nStims0])
ax[2].plot(nStims0, meanAreasOfCell2); ax[2].set_title('C. Mean contribution of the 2nd cell to the area, p(MEP) = 0.7')
ax[2].grid(); ax[2].set_xlabel('Number of stimuli per cell'); ax[2].set_ylabel('Area estimate')
setSize(8,10)
plt.subplots_adjust(hspace = 0.5)
plt.savefig('14 Non-monotonic bias function for a model map.png', dpi = 300, bbox_inches = 'tight')
plt.close(plt.gcf())

In [73]:
# Spurious area difference due to the bias: a model representation with the same true area (=1),
# but different mean estimates
pTrue1 = np.array([0.1,0.6]); areaTrue1 = np.sum(pTrue>pThr)
meanAreas1 = np.array([np.sum([pBinom(pThr,nStim, p) for p in pTrue1]) for nStim in nStims0])
plt.plot(nStims0[:10], meanAreas[:10]);
plt.plot(nStims0[:10], meanAreas1[:10]); plt.ylim(0,1.2)
plt.xlabel('Number of stimuli per cell'); plt.ylabel('Mean area estimate');
plt.legend(['First representation, p(MEP)=0.4, 0.7', 'Second representation, p(MEP)=0.1, 0.6']);
plt.savefig('Spurious area difference due to the bias.png', dpi = 300, bbox_inches = 'tight')
plt.close(plt.gcf())

In [22]:
meanAreas[4]

1.15436

In [23]:
meanAreas1[4]

0.6911199999999997

In [74]:
# The spurious difference is eliminated only at large numbers of stimuli per cell
plt.plot(nStims0, meanAreas);
plt.plot(nStims0, meanAreas1); plt.ylim(0,1.2)
plt.xlabel('Number of stimuli per cell'); plt.ylabel('Mean area estimate');
plt.legend(['First representation, p(MEP)=0.4, 0.7', 'Second representation, p(MEP)=0.1, 0.6']);
plt.savefig('Spurious area difference due to the bias, up to 100 stim.png', dpi = 300, bbox_inches = 'tight')
plt.close(plt.gcf())

In [75]:
meanAreas[98]

1.0219148943303238

In [76]:
meanAreas1[98]

0.9780695578699149

## <font color=green> **Bias of area within sessions**</font>

In [27]:
# calculate and plot the relative bias values of the representation parameters
# for the bootstrapping-generated maps with respect to the values from the initial maps

In [28]:
meanMEPthrAreaRelBias = paramRelBiasArr(meanMEPthrArea, boots, TMSmaps)
maxMEPthrAreaRelBias = paramRelBiasArr(maxMEPthrArea, boots, TMSmaps)
probHalfThrAreaRelBias = paramRelBiasArr(probHalfThrArea, boots, TMSmaps)
meanWareaRelBias = paramRelBiasArr(meanWarea, boots, TMSmaps)
probWareaRelBias = paramRelBiasArr(probWarea, boots, TMSmaps)

In [77]:
f,ax = plt.subplots(); plt.sca(ax)
color = next(ax._get_lines.prop_cycler)['color']
allMapsMedianPlot(meanMEPthrAreaRelBias, col = color)
color = next(ax._get_lines.prop_cycler)['color']
allMapsMedianPlot(maxMEPthrAreaRelBias, col = color)
color = next(ax._get_lines.prop_cycler)['color']
allMapsMedianPlot(probHalfThrAreaRelBias, isEvenNstim = True, col = color)
allMapsMedianPlot(probHalfThrAreaRelBias, isOddNstim = True, col = color, linestyle = '--')
color = next(ax._get_lines.prop_cycler)['color']
allMapsMedianPlot(meanWareaRelBias, col = color)
color = next(ax._get_lines.prop_cycler)['color']
allMapsMedianPlot(probWareaRelBias, col = color)
plt.legend(['Area of cells with mean MEP > 50 $\mu$V',
            'Area of cells with max MEP > 50 $\mu$V',
            'Area of cells with more than half MEPs > 50 $\mu$V (even n. stim.)',
            'Area of cells with more than half MEPs > 50 $\mu$V (odd n. stim.)',
            'Area weighted by mean MEP','Area weighted by probability of MEP > 50 $\mu$V'],
           loc = 'lower right')
plt.ylabel('Normalized bias')
plt.xlim(0,11); plt.xticks(range(1,11))
#plt.ylim(-0.65,0.1);
plt.ylim(-0.65,0.4);
setSize(7.5,6.5); plt.grid()
plt.tight_layout()
plt.savefig('03 Within-session bias for area parameters.png', dpi = 300)
plt.close(plt.gcf())

In [30]:
# Illustration of bias heterogeneity
# Plotting biases for individual mapping sessions in all subjects

In [78]:
for iSubj in iSubjs:
    for iSess in iSessions:
        plt.plot(nStims, meanMEPthrAreaRelBias[nStims,iSubj,iSess])
plt.xlabel('Number of stimuli per grid cell'); plt.ylabel('Normalized bias')
plt.xlim(0,11); plt.grid(); plt.ylim(None,0.3);
plt.tight_layout(); 
plt.savefig('09 Within-session bias for mean MEP-thresholded area.png', dpi = 300)
plt.close(plt.gcf())

In [79]:
for iSubj in iSubjs:
    for iSess in iSessions:
        plt.plot(nStims, maxMEPthrAreaRelBias[nStims,iSubj,iSess])
plt.xlabel('Number of stimuli per grid cell'); plt.ylabel('Normalized bias')
plt.xlim(0,11); plt.grid(); plt.ylim(None,0.1);
plt.tight_layout(); 
plt.savefig('10 Within-session bias for max MEP-thresholded area.png', dpi = 300)
plt.close(plt.gcf())

In [80]:
for iSubj in iSubjs:
    for iSess in iSessions:
        plt.plot(nStims, probHalfThrAreaRelBias[nStims,iSubj,iSess])
plt.xlabel('Number of stimuli per grid cell'); plt.ylabel('Normalized bias')
plt.xlim(0,11); plt.grid();  plt.ylim(None,2.1);
plt.tight_layout(); 
plt.savefig('11 Within-session bias for area with more than half suprathreshold MEPs.png', dpi = 300)
plt.close(plt.gcf())

In [81]:
for iSubj in iSubjs:
    for iSess in iSessions:
        plt.plot(nStims, meanWareaRelBias[nStims,iSubj,iSess])
plt.xlabel('Number of stimuli per grid cell'); plt.ylabel('Normalized bias')
plt.xlim(0,11); plt.grid();  plt.ylim(None,None);
plt.tight_layout(); 
plt.savefig('12 Within-session bias for mean MEP-weighted area.png', dpi = 300)
plt.close(plt.gcf())

In [82]:
for iSubj in iSubjs:
    for iSess in iSessions:
        plt.plot(nStims, probWareaRelBias[nStims,iSubj,iSess])
plt.xlabel('Number of stimuli per grid cell'); plt.ylabel('Normalized bias')
plt.xlim(0,11); plt.grid();  plt.ylim(None,None);
plt.tight_layout(); 
plt.savefig('13 Within-session bias for probability-weighted area.png', dpi = 300)
plt.close(plt.gcf())

## <font color=green> **CV of area within sessions**</font>

In [36]:
# calculate and plot the CV of the representation parameters for the bootstrapping-generated maps

In [37]:
meanMEPthrAreaCV = paramCVarr(meanMEPthrArea, boots)
maxMEPthrAreaCV = paramCVarr(maxMEPthrArea, boots)
probHalfThrAreaCV = paramCVarr(probHalfThrArea, boots)
meanWareaCV = paramCVarr(meanWarea, boots)
probWareaCV = paramCVarr(probWarea, boots)

In [83]:
lastNstim = 10
f,ax = plt.subplots(); plt.sca(ax)
color = next(ax._get_lines.prop_cycler)['color']
allMapsMedianPlot(meanMEPthrAreaCV, col = color)
color = next(ax._get_lines.prop_cycler)['color']
allMapsMedianPlot(maxMEPthrAreaCV, col = color)
color = next(ax._get_lines.prop_cycler)['color']
allMapsMedianPlot(probHalfThrAreaCV, isEvenNstim = True, col = color)
allMapsMedianPlot(probHalfThrAreaCV, isOddNstim = True, col = color, linestyle = '--')
color = next(ax._get_lines.prop_cycler)['color']
allMapsMedianPlot(meanWareaCV, col = color)
color = next(ax._get_lines.prop_cycler)['color']
allMapsMedianPlot(probWareaCV, col = color)
plt.legend(['Area of cells with mean MEP > 50 $\mu$V',
            'Area of cells with max MEP > 50 $\mu$V', 
            'Area of cells with more than half MEPs > 50 $\mu$V (even n. stim.)',
            'Area of cells with more than half MEPs > 50 $\mu$V (odd n. stim.)',
            'Area weighted by mean MEP',
            'Area weighted by probability of MEP > 50 $\mu$V'],
           loc = 'upper right')
plt.ylabel('CV within session')
plt.xlim(0,nStimBoot+1); plt.ylim(0,0.6); plt.xticks(range(1,nStimBoot+1))
setSize(7,6); plt.grid()
plt.tight_layout()
plt.savefig('04 Within-session CV for area parameters.png', dpi = 300)
plt.close(plt.gcf())

In [39]:
# test for decrease with the number of stimuli
print('meanMEPthrAreaCV -', PageAndFriedmanTestsForMeansBySessions(meanMEPthrAreaCV))
print('maxMEPthrAreaCV -', PageAndFriedmanTestsForMeansBySessions(maxMEPthrAreaCV))
print('probHalfThrAreaCV -', PageAndFriedmanTestsForMeansBySessions(probHalfThrAreaCV, isEvenNstim = True))
print('probWareaCV -', PageAndFriedmanTestsForMeansBySessions(probWareaCV))
print('meanWareaCV -', PageAndFriedmanTestsForMeansBySessions(meanWareaCV))

meanMEPthrAreaCV - Page: "<=.001", Friedman: 6.2e-12
maxMEPthrAreaCV - Page: "<=.001", Friedman: 6.2e-12
probHalfThrAreaCV - Page: "<=.001", Friedman: 1.9e-06
probWareaCV - Page: "<=.001", Friedman: 6.2e-12
meanWareaCV - Page: "<=.001", Friedman: 6.2e-12


## <font color=green> **Total within-session accuracy**</font>

In [40]:
meanMEPthrAreaRMSDs = paramRelRMSDarr(meanMEPthrArea, boots, TMSmaps)
maxMEPthrAreaRMSDs = paramRelRMSDarr(maxMEPthrArea, boots, TMSmaps)
probHalfThrAreaRMSDs = paramRelRMSDarr(probHalfThrArea, boots, TMSmaps)
probWareaRMSDs = paramRelRMSDarr(probWarea, boots, TMSmaps)
meanWareaRMSDs = paramRelRMSDarr(meanWarea, boots, TMSmaps)

In [41]:
# Total accuracies for nStim = 10
[np.mean(meanMEPthrAreaRMSDs[10]), np.mean(maxMEPthrAreaRMSDs[10]),np.mean(probHalfThrAreaRMSDs[10]),
 np.mean(meanWareaRMSDs[10]),np.mean(probWareaRMSDs[10])]

[0.14354036048423427,
 0.10655321468261995,
 0.36471333510179743,
 0.15836387351257794,
 0.07980229538709084]

In [42]:
# Total accuracies averaged by all nStim
[np.mean(meanMEPthrAreaRMSDs), np.mean(maxMEPthrAreaRMSDs),np.mean(probHalfThrAreaRMSDs),
 np.mean(meanWareaRMSDs),np.mean(probWareaRMSDs)]

[0.1860379978224549,
 0.22800475031297426,
 0.43998172357484183,
 0.22924269218469684,
 0.1150624266251459]

## <font color=green> **Variability of area between sessions**</font>

In [43]:
# calculate and plot the the values of the between-session variability index
# (defined as half the relative difference between max and min among the sessions)

In [44]:
meanMEPthrAreaVar = paramHalfRelDiffArr(meanMEPthrArea, boots)
maxMEPthrAreaVar = paramHalfRelDiffArr(maxMEPthrArea, boots)
probHalfThrAreaVar = paramHalfRelDiffArr(probHalfThrArea, boots)
meanWareaVar = paramHalfRelDiffArr(meanWarea, boots)
probWareaVar = paramHalfRelDiffArr(probWarea, boots)

In [84]:
f,ax = plt.subplots(); plt.sca(ax)
color = next(ax._get_lines.prop_cycler)['color']
medianPlot(meanMEPthrAreaVar, col = color)
color = next(ax._get_lines.prop_cycler)['color']
medianPlot(maxMEPthrAreaVar, col = color)
color = next(ax._get_lines.prop_cycler)['color']
medianPlot(probHalfThrAreaVar, isEvenNstim = True, col = color)
medianPlot(probHalfThrAreaVar, isOddNstim = True, col = color, linestyle = '--')
color = next(ax._get_lines.prop_cycler)['color']
medianPlot(meanWareaVar, col = color)
color = next(ax._get_lines.prop_cycler)['color']
medianPlot(probWareaVar, col = color)
plt.legend(['Area of cells with mean MEP>50$\mu$V',
            'Area of cells with max MEP>50$\mu$V',
            'Area of cells with more than half MEPs > 50 $\mu$V (even n. stim.)',
            'Area of cells with more than half MEPs > 50 $\mu$V (odd n. stim.)',
            'Area weighted by mean MEP', 'Area weighted by probability of MEP > 50$\mu$V'])
plt.ylim(0,0.8);
setSize(7,7); plt.grid()
plt.ylabel('Between-session variability index')
plt.xticks(range(1,11));
plt.savefig('05 Between-session variability vs num stim.png', dpi = 300, bbox_inches = 'tight')
plt.close(plt.gcf())

In [46]:
# test for decrease with the number of stimuli
print('meanMEPthrAreaVar -', PageAndFriedmanTests(meanMEPthrAreaVar))
print('maxMEPthrAreaVar -', PageAndFriedmanTests(maxMEPthrAreaVar))
print('probHalfThrAreaVar -', PageAndFriedmanTests(probHalfThrAreaVar, isEvenNstim = True))
print('meanWareaVar -', PageAndFriedmanTests(meanWareaVar))
print('probWareaVar -', PageAndFriedmanTests(probWareaVar))

meanMEPthrAreaVar - Page: "<=.001", Friedman: 1.0e-05
maxMEPthrAreaVar - Page: "<=.001", Friedman: 5.8e-07
probHalfThrAreaVar - Page: "<=.001", Friedman: 1.9e-05
meanWareaVar - Page: "<=.001", Friedman: 1.6e-06
probWareaVar - Page: "<=.001", Friedman: 1.1e-11


In [47]:
print('probHalfThrAreaVar, odd nStim -', PageAndFriedmanTests(probHalfThrAreaVar, isOddNstim = True))

probHalfThrAreaVar, odd nStim - Page: "NS", Friedman: 8.6e-01


## <font color=green> **Distributions of probability-weighted area in sessions**</font>

In [48]:
# Compute and plot the distributions of the probability-weighted area in the maps generated by
# boostrapping from each session of each subject.
# The subplot titles show the distribution overlaps
# and the values of the bootstrapping-based between-session intraclass correlation coefficient,
# which measures the ability to discriminate between sessions in the presence of within-session inaccuracy

In [85]:
probWareaArr = paramArrForNstim(probWarea, boots)
plotParamHists(probWareaArr, probWarea, TMSmaps, xlab = 'Area weighted by probability of MEP>50 $\mu$V, $mm^2$', unit = cellArea)

plt.savefig('06 Histograms with BICC and overlaps.png', dpi = 300)
plt.close(plt.gcf())

In [50]:
# Plotting BICC as a function of the number of stimuli per cell

In [51]:
probWareaArrs = np.array([paramArrForNstim(probWarea, boots, takeFirst=nSt) for nSt in nStims])

In [52]:
BICCs = np.array([np.median([ICConeWay(subj) for subj in arr]) for arr in probWareaArrs])

In [87]:
plt.plot(nStims, BICCs,'o-')
plt.xlabel('Number of stimuli per grid cell'); plt.xlim(0,11)
plt.ylabel('BICC'); plt.ylim(0,1)
plt.grid();
plt.savefig('BICC as a function of the number of stimuli per cell.png', dpi = 300)
plt.close(plt.gcf())

In [54]:
# - detailed mapping is able to find significant area changes at the individual level

## <font color=green> **Bootstrapping-based between-session ICC for area variants**</font>

In [55]:
meanMEPthrAreaArr = paramArrForNstim(meanMEPthrArea, boots)
maxMEPthrAreaArr = paramArrForNstim(maxMEPthrArea, boots)
probHalfThrAreaArr = paramArrForNstim(probHalfThrArea, boots)
meanWareaArr = paramArrForNstim(meanWarea, boots)

In [56]:
def ICCarr(arrs):
    return np.array([ICConeWay(subj) for subj in arrs])

In [88]:
plt.plot(iSubjs+1, ICCarr(meanMEPthrAreaArr));
plt.plot(iSubjs+1, ICCarr(maxMEPthrAreaArr));
plt.plot(iSubjs+1, ICCarr(probHalfThrAreaArr));
plt.plot(iSubjs+1, ICCarr(meanWareaArr));
plt.plot(iSubjs+1, ICCarr(probWareaArr));
plt.legend(['Area of cells with mean MEP>50$\mu$V',
            'Area of cells with max MEP>50$\mu$V',
            'Area of cells with more than half MEPs > 50 $\mu$V',
            'Area weighted by mean MEP', 'Area weighted by probability of MEP>50$\mu$V'])
plt.ylim(0,1.1);
setSize(7,5); plt.grid()
plt.xlabel('Subject number'); plt.ylabel('BICC')
plt.savefig('07 BICC for all parameters.png', dpi = 300, bbox_inches = 'tight')
plt.close(plt.gcf())

## <font color=green> **Accuracy of COGs within sessions**</font>

In [58]:
# compute and plot the accuracy of the COGs with 3 alternative weight definitions
# (the accuracy is defined as the mean distance between the COGs of the bootstrapping-generated maps
# and the COGs of the initial maps)

In [59]:
distCOGbyMean = vecParamMeanDistFromTrue(COGbyMean, boots, TMSmaps)
distCOGbyMax = vecParamMeanDistFromTrue(COGbyMax, boots, TMSmaps)
distCOGbyProb = vecParamMeanDistFromTrue(COGbyProb, boots, TMSmaps)

In [89]:
p1 = allMapsMedianPlot(cellSize * distCOGbyMean, label = 'COG weighted by mean MEP amplitude')
p2 = allMapsMedianPlot(cellSize * distCOGbyMax, label = 'COG weighted by max MEP amplitude')
plt.plot([],[]) # halfProb
plt.plot([],[]) # mean-w
p3 = allMapsMedianPlot(cellSize * distCOGbyProb, label = 'COG weighted by probability of MEP>50 $\mu$V')
plt.legend()
plt.ylabel('Mean error, mm')
plt.ylim(0,4); plt.xticks(range(11)); plt.xlim(0,11)
setSize(6,4); plt.grid(); 
plt.tight_layout()
plt.savefig('08 Within-session errors in COG.png', dpi = 300)
plt.close(plt.gcf())

In [61]:
# test for decrease with the number of stimuli
print('distCOGbyMean -', PageAndFriedmanTestsForMeansBySessions(distCOGbyMean))
print('distCOGbyMax -', PageAndFriedmanTestsForMeansBySessions(distCOGbyMax))
print('distCOGbyProb -', PageAndFriedmanTestsForMeansBySessions(distCOGbyProb))

distCOGbyMean - Page: "<=.001", Friedman: 6.2e-12
distCOGbyMax - Page: "<=.001", Friedman: 6.2e-12
distCOGbyProb - Page: "<=.001", Friedman: 6.2e-12


In [62]:
# test for differences between the three COG definitions
# the accuracies are averaged by the three sessions in each subject to obtain independent values
for nStim in range(1,11):
    print('%2.1e'%scipy.stats.friedmanchisquare(np.mean(distCOGbyMean[nStim], axis = 1),
                                                np.mean(distCOGbyMax[nStim], axis = 1),
                                                np.mean(distCOGbyProb[nStim], axis = 1))[1])

3.4e-04
3.4e-04
8.0e-04
8.0e-04
8.0e-04
2.1e-02
3.4e-02
3.4e-02
9.3e-02
2.0e-01
